## Setup and Loading Data

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')

In [2]:
train = pd.read_csv("../input/2020aibootacampqualifiers/Train.csv", index_col="Applicant_ID")

train.head(2)

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
Applicant_ID,,,,,,,,,,,,,,,,,,,,,
Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.0,0.0,10689720.0,252072.0,4272776.0,11333126.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.0,0.0,898979.0,497531.0,9073814.0,2533168.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.80573,1.620483,0.322436,no


In [3]:
test = pd.read_csv("../input/2020aibootacampqualifiers/Test.csv", index_col="Applicant_ID")

test.head(2)

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
Applicant_ID,,,,,,,,,,,,,,,,,,,,,
Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,0.0,...,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,18308285.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525


## Data Preprocessing

### Checking & Encoding Categorical Columns

In [4]:
train.default_status = train.default_status.map({'no': 0, 'yes': 1})

In [5]:
weight = 300
global_mean = train.default_status.mean()

# compute the number of values and mean of the column
aggs = train.groupby('form_field47')['default_status'].agg(['count', 'mean'])
counts, means = aggs['count'], aggs['mean']

# compute the smoothed means
smooth = (counts * means + weight + global_mean) / (counts + weight)

train.form_field47 = train.form_field47.map(smooth)
test.form_field47 = test.form_field47.map(smooth)

In [6]:
train.head(2)

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
Applicant_ID,,,,,,,,,,,,,,,,,,,,,
Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.0,0.0,10689720.0,252072.0,4272776.0,11333126.0,...,0.392854,2.02,0.711632,0.0,0.0,0.322506,NaN,1.129518,0.044335,0
Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.0,0.0,898979.0,497531.0,9073814.0,2533168.0,...,0.314281,8.08,0.183584,NaN,0.0,0.322506,349.80573,1.620483,0.322436,0


### Separating the Target from the predictors

In [7]:
y = train.pop("default_status")

train.head(2)

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
Applicant_ID,,,,,,,,,,,,,,,,,,,,,
Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.0,0.0,10689720.0,252072.0,4272776.0,11333126.0,...,NaN,0.392854,2.02,0.711632,0.0,0.0,0.322506,NaN,1.129518,0.044335
Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.0,0.0,898979.0,497531.0,9073814.0,2533168.0,...,NaN,0.314281,8.08,0.183584,NaN,0.0,0.322506,349.80573,1.620483,0.322436


### Handling Missing Values

In [8]:
from sklearn.impute import SimpleImputer

zero_imputer = SimpleImputer(strategy='constant', add_indicator=True)

In [9]:
imputed_train = zero_imputer.fit_transform(train)
imputed_test = zero_imputer.transform(test)

## Scaling the data

In [10]:
from sklearn.preprocessing import MinMaxScaler

In [11]:
scaler_mm = MinMaxScaler()

scaled_train = pd.DataFrame(scaler_mm.fit_transform(imputed_train))
scaled_test = pd.DataFrame(scaler_mm.transform(imputed_test))

## Modelling

### Hyperparameter Tuning

XGBoost: learning_rate (0.01 - 0.2), max_depth, min_child_weight, colsample_bytree, subsample, n_estimators

CatBoost: learning_rate, depth, l2-leaf-reg, rsm, iterations

LightGBM: learning_rate, max_depth, num_leaves, min_data_in_leaf(min_data, min_child_samples), feature_fraction, bagging_fraction, num_iterations

In [12]:
!pip uninstall typing -y
!pip install tune-sklearn scikit-optimize hyperopt hpbandster

Found existing installation: typing 3.7.4.3
Uninstalling typing-3.7.4.3:
  Successfully uninstalled typing-3.7.4.3
     |████████████████████████████████| 51 kB 53 kB/s 
     |████████████████████████████████| 89 kB 830 kB/s 
     |████████████████████████████████| 964 kB 2.3 MB/s 
  Installing build dependencies ... - \ | / - \ | done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... - \ done
  Created wheel for hpbandster: filename=hpbandster-0.7.4-py3-none-any.whl size=80006 sha256=634e1c956aa9a4e68661951f0b21c91d298544dbad145821ee9f9de4defec5ac
  Stored in directory: /root/.cache/pip/wheels/d9/88/fc/61ab6b9f386a386839668631c39a6dc3c2fb0ec7000d552faa
  Created wheel for ConfigSpace: filename=ConfigSpace-0.4.15-cp37-cp37m-linux_x86_64.whl size=2890991 sha256=6be0ca5904d21d6e3027ae1e1e073ec81e8fc516ade312be94a0d11318ffda83
  Stored in directory: /root/.cache/pip/wheels/d2/5b/52/78dc4cbd8f56c71a50b9a19ef34b23147664c3c726233b25e9
  Created

In [13]:
from tune_sklearn import TuneSearchCV, TuneGridSearchCV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from rgf.sklearn import RGFClassifier, FastRGFClassifier
from sklearn.ensemble import RandomForestClassifier

In [14]:
xgb_params = {
    'max_depth': [3,4,5,6,7,8,9,10],
    'min_child_weight': [1,2,3,5],
    'gamma': [0,.1,.2,.3,.4,.5,1,1.5,2,5],
    'subsample': [.5,.6,.7,.8,.9],
    'colsample_bytree': [.5,.6,.7,.8,.9],
    'scale_pos_weight': [1, .5, .8, 1.5],
    'learning_rate': [0.03, 0.001, 0.005, 0.01, 0.1, 0.2, 0.3],
    'n_estimators': [250, 100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500],
    'max_depth': [3, 1, 2, 4, 5, 6, 7, 8, 9, 10],
    'min_child_weight': [3,4,5,6,7,8,9,10],
    'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05],
    'seed': [7, 50, 300, 800, 1200, 1800, 2400, 3000],
}

xgb_model = XGBClassifier(metrics='auc', nthread=-1)

tune_xgb_1 = TuneGridSearchCV(xgb_model, 
                          xgb_params,
                          max_iters=15,
                              use_gpu=True,
                         # n_jobs=-1)

SyntaxError: unexpected EOF while parsing (<ipython-input-14-c6671f0ee172>, line 22)

In [15]:
tune_xgb_1.fit(scaled_train, y)

NameError: name 'tune_xgb_1' is not defined

In [16]:
tune_xgb_1.best_params_

NameError: name 'tune_xgb_1' is not defined

In [17]:
tune_xgb_1.cv_results_

NameError: name 'tune_xgb_1' is not defined

In [18]:
# cat_params = {
#     'depth': [3, 1, 2, 4, 5, 6, 7, 8, 9, 10],
#     'iterations': [250, 100, 500, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500],
#     'learning_rate': [0.03, 0.001, 0.005, 0.01, 0.1, 0.2, 0.3],
#     'l2_leaf_reg': [3, 1, 5, 10, 100],
#     'border_count': [32, 5, 10, 20, 50, 100, 200],
#     'ctr_border_count': [50, 5, 10, 20, 100, 200],
# }

# cat_model = CatBoostClassifier()
# search_results = cat_model.randomized_search(cat_params, scaled_train, y, cv=5, plot=True)

### Local Validation

In [19]:
model1 = XGBClassifier(**tune_xgb_1.best_params_)
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=99)

NameError: name 'tune_xgb_1' is not defined

In [20]:
local_validation = []
test_predictions = []

for i, (train_index, val_index) in enumerate(kf.split(scaled_train, y)):
    print("*" * 75 + "\nFold" + str(i))
    cv_train, cv_val = scaled_train.iloc[train_index], scaled_train.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    model1.fit(cv_train, y_train)
    y_pred = model1.predict_proba(cv_val)[:, 1]
    local_validation.append(roc_auc_score(y_val, y_pred))
    
    test_pred = model1.predict_proba(scaled_test)[:, 1]
    test_predictions.append(test_pred)   

NameError: name 'kf' is not defined

In [21]:
print(local_validation)
np.mean(local_validation)

[]


/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan

In [22]:
test_pred = np.mean(test_predictions, axis=0)

### Final Steps

In [23]:
submission = pd.read_csv("../input/2020aibootacampqualifiers/SampleSubmission.csv")

In [24]:
submission.default_status = test_pred
submission.to_csv("sub_xgb", index=False)

In [25]:
submission.head(2)

,Applicant_ID,default_status
0,Apcnt_1000032,NaN
1,Apcnt_1000048,NaN


### Next Steps

How do we improve our model's accuracy?